# Project Title: Student Performance Prediction Using Decision Tree Classifier

Author: Md. Maruf
Date: 26-12-2025

Project Overview:
This project aims to predict the performance of students based on various features such as 
Student_ID,Age,Gender,Class,Study_Hours_Per_Day,Attendance_Percentage,and other relevant attributes. The main goal is to classify students into different performance categories using a Decision Tree Classifier.

Dataset:
- Source: Dataset downloaded from kaggle.
- Features: Student_ID,Age,Gender,Class,Study_Hours_Per_Day,Attendance_Percentage,       Parental_Education,Internet_Access,Extracurricular_Activities,Math_Score,Science_Score,English_Score,Previous_Year_Score,Final_Percentage,Performance_Level,Pass_Fail.
- Target: Student performance category (Pass or Fail)

Objectives:
1. Load and explore the student performance dataset.
2. Preprocess the data (handle missing values, encode categorical variables, normalize if needed).
3. Train a Decision Tree Classifier model.
4. Evaluate the model's performance using metrics like accuracy, recall, and F1-score, Classification report, Confussion Metrics, ruc auc score.
5. Visualize the decision tree for better understanding and interpretation.

Tools & Libraries:
- Python 3.x
- pandas, numpy
- scikit-learn
- matplotlib, seaborn (for visualization)

Expected Outcome:
- A trained Decision Tree model capable of predicting student performance.
- Insights into which features most influence student performance.
- Visual representation of the decision-making process of the tree.

Notes:
- Ensure to split the dataset into training and testing sets to avoid overfitting.
- Hyperparameter tuning may be applied to improve model accuracy.


In [1]:
# Importing Necessary libraries
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score, f1_score, recall_score, precision_score, roc_auc_score, auc, roc_curve

In [ ]:
# Load the student performance dataset from a CSV file into a pandas DataFrame
df = pd.read_csv('Student_Performance_Dataset.csv')


In [6]:
## Display the first five rows of the DataFrame to quickly inspect the dataset structure and values
df.head()

,Student_ID,Age,Gender,Class,Study_Hours_Per_Day,Attendance_Percentage,Parental_Education,Internet_Access,Extracurricular_Activities,Math_Score,Science_Score,English_Score,Previous_Year_Score,Final_Percentage,Performance_Level,Pass_Fail
0,S0001,15,Male,12,1.0,65,Postgraduate,No,No,40,39,72,81,50.33,Average,Pass
1,S0002,19,Female,9,1.6,58,Graduate,No,Yes,80,44,35,47,53.00,Average,Pass
2,S0003,14,Female,12,3.6,64,High School,Yes,Yes,83,73,59,58,71.67,Good,Pass
3,S0004,18,Female,9,5.5,68,Postgraduate,Yes,No,68,48,77,54,64.33,Average,Pass
4,S0005,14,Male,10,5.0,80,High School,Yes,No,41,46,36,68,41.00,Poor,Fail


In [7]:
# Display a concise summary of the DataFrame including column names, data types, and non-null counts
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 16 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   Student_ID                  5000 non-null   object 
 1   Age                         5000 non-null   int64  
 2   Gender                      5000 non-null   object 
 3   Class                       5000 non-null   int64  
 4   Study_Hours_Per_Day         5000 non-null   float64
 5   Attendance_Percentage       5000 non-null   int64  
 6   Parental_Education          5000 non-null   object 
 7   Internet_Access             5000 non-null   object 
 8   Extracurricular_Activities  5000 non-null   object 
 9   Math_Score                  5000 non-null   int64  
 10  Science_Score               5000 non-null   int64  
 11  English_Score               5000 non-null   int64  
 12  Previous_Year_Score         5000 non-null   int64  
 13  Final_Percentage            5000 

In [10]:
# Return the number of rows and columns in the DataFrame
df.shape

(5000, 16)

In [12]:
# Check for missing values in each column and return the total count of null values per column
df.isnull().sum()

Student_ID                    0
Age                           0
Gender                        0
Class                         0
Study_Hours_Per_Day           0
Attendance_Percentage         0
Parental_Education            0
Internet_Access               0
Extracurricular_Activities    0
Math_Score                    0
Science_Score                 0
English_Score                 0
Previous_Year_Score           0
Final_Percentage              0
Performance_Level             0
Pass_Fail                     0
dtype: int64

In [ ]:
sel_col = df[['Age', 'Class']]